In [14]:
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from PIL import Image
from skimage import feature
import numpy as np

# Load CIFAR-10 dataset
(ds_train, ds_test), ds_info = tfds.load('cifar10', split=['train', 'test'], shuffle_files=True, with_info=True, as_supervised=True)

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)

# Convert dataset to numpy arrays
train_images = []
train_labels = []

for data in tfds.as_dataframe(ds_train, ds_info):
    train_images.append(data[0])
    train_labels.append(data[1])

test_images = []
test_labels = []

for data in tfds.as_dataframe(ds_test, ds_info):
    test_images.append(data[0])
    test_labels.append(data[1])

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Compute SIFT features
train_sift_features = []
test_sift_features = []

for i in range(train_images.shape[0]):
    image = train_images[i]
    print(type())
    # image = np.array(image).astype(np.float32)
    image = (image * 255).astype(np.uint8)
    image = Image.fromarray(image)
    sift_features = feature.local_binary_pattern(image, 8, 1, method="nri_uniform")
    train_sift_features.append(sift_features)

for i in range(test_images.shape[0]):
    image = test_images[i]
    image = np.array(image).astype(np.float32)
    image = (image * 255).astype(np.uint8)
    image = Image.fromarray(image)
    sift_features = feature.local_binary_pattern(image, 8, 1, method="nri_uniform")
    test_sift_features.append(sift_features)

train_sift_features = np.array(train_sift_features)
test_sift_features = np.array(test_sift_features)

# Flatten SIFT features
train_sift_features = train_sift_features.reshape((train_sift_features.shape[0], -1))
test_sift_features = test_sift_features.reshape((test_sift_features.shape[0], -1))

# Train SVC model
clf = svm.SVC(kernel='linear', C=1)
clf.fit(train_sift_features, train_labels)

# Evaluate SVC model
predictions = clf.predict(test_sift_features)

print("Classification Report:")
print(classification_report(test_labels, predictions))

print("Confusion Matrix:")
print(confusion_matrix(test_labels, predictions))

print("Accuracy Score:")
print(accuracy_score(test_labels, predictions))

ValueError: could not convert string to float: 'i'